# Terraform

Terraform is an infrastructure-as-code (IaC) tool (created by HashiCorp) which permits users to define infrastructure using a declarative configuration language known as HashiCorp Configuration Language (HCL).

- Terraform Install:
  - https://developer.hashicorp.com/terraform/install
- Terraform Language Documentation:
  - https://developer.hashicorp.com/terraform/language
  - https://developer.hashicorp.com/terraform/language/providers
- Terraform Providers
  - https://registry.terraform.io/browse/providers
  - https://registry.terraform.io/providers/hashicorp/azurerm/latest
  - https://registry.terraform.io/providers/hashicorp/kubernetes/latest

## Show Terraform version

In [1]:
!terraform --version

Terraform v1.7.2
on linux_amd64

Your version of Terraform is out of date! The latest version
is 1.7.3. You can update by downloading from https://www.terraform.io/downloads.html


## Show Terraform help

- The command `terraform --help` shows help about terraform commands.
- The command `terraform [command] --help` shows additional help about a specific command, e.g. `terraform apply --help`.

In [2]:
!terraform --help

Usage: terraform [global options] <subcommand> [args]

The available commands for execution are listed below.
The primary workflow commands are given first, followed by
less common or more advanced commands.

Main commands:
  init          Prepare your working directory for other commands
  validate      Check whether the configuration is valid
  plan          Show changes required by the current configuration
  apply         Create or update infrastructure
  destroy       Destroy previously-created infrastructure

All other commands:
  console       Try Terraform expressions at an interactive command prompt
  fmt           Reformat your configuration in the standard style
  force-unlock  Release a stuck lock on the current workspace
  get           Install or upgrade remote Terraform modules
  graph         Generate a Graphviz graph of the steps in an operation
  import        Associate existing infrastructure with a Terraform resource
  login         Obtain and save credentials for a

## Terraform files

- Terraform infrastructure is defined in Terraform files with the `.tf` file extension.
- A folder can contain one or more `.tf` files.
  - Usually, infrastructure definitions are spread out among multiple `.tf` files.
- Terraform commands are issued within a specific folder containing `.tf` files.
  - All `.tf` files within that folder will be merged into one *file* in memory (RAM) beore the command interprets the code.

#### In this example, the folder we will be issuing Terraform commands against contains three `.tf` files.

In [3]:
#dir *.tf # use this on Windows
!ls *.tf

providers.tf  resource-group.tf  variables.tf


## Terraform Hashicorp Configuration Language (HCL)

- Each `.tf` contains declarations using the Hashicorp Configuration Language (https://developer.hashicorp.com/terraform/language).
- Hashicorp Configuration Language (HCL) basically consists of `Blocks`, `Arguments` and `Expressions`.

  ```bash
  <BLOCK TYPE> "<BLOCK LABEL>" "<BLOCK LABEL>" {
    # Block body
    <IDENTIFIER> = <EXPRESSION> # Argument
  }
  ```

  - `Blocks`
    - Blocks are containers for other content and usually represent the configuration of some kind of object, like a resource.
    - Blocks have a `<BLOCK TYPE>`, can have zero or more `<BLOCK LABEL>`s.
    - A Block has a body between `{}` that contains any number of arguments and nested blocks.
    - Most of Terraform's features are controlled by top-level blocks in a configuration file.

  - `Arguments`
    - Arguments assign a value to a name, i.e. an `<EXPRESSION>` to an `<IDENTIFIER>`.
    - They appear within a Block body.

  - `Expressions`
    - Expressions represent a value, either literally or by referencing and combining other values.
    - They appear as values for arguments, or within other expressions.
    
  - `Comments`
    - The hash character `#` is used for creating line comments in HCL.

- Here's a concrete example:
  
  ```bash
  # Define an Azure Resource Group
  resource "azurerm_resource_group" "main" {
    name     = "tsfn14g00"
    location = "westeurope"
  }
  ```

  - `Block`
    - `resource` is the `<BLOCK_TYPE>` which means the Block is defining a **resource**.
    - `"azurerm_resource_group"` is the first `<BLOCK_LABEL>` which states what resource we are defining (an Azure Resource Group).
    - `"main"` is the second `<BLOCK_LABEL>` whcih is a name we choose to uniquely identify this Block within our Terraform files.
  - `Arguments`
    - `name = "tsfn14g00"` is the first argument that assigns the value (expression) `"tsfn14g00"` to the name (identifier) `name`.
    - `location = "westeurope"` is the second argument that assigns the value (expression) `"westeurope"` to the name (identifier) `location`.

## Let's view the contents of the file  `providers.tf`

- This file configures **Terraform providers**.
  - A **provider** can be seen as a **Terraform plugin** with support for defining **resources** (infrastructure) for that specific provider.
  - In this case, only one provier `azurerm` is configured, providing functionality for defining **Azure** resources (infrastructure).

- Hashicorp Configuration Language (HCL)
  - At the root level, the file contains two blocks with block types `terraform` and `provider`.
  - The `terraform` block is used to configure the `terraform` CLI tool, and contains:
    - A nested block `required_providers` used to specify Terraform providers (plugins) to use, which contains one argument:
      - `azurerm = { source = "hashicorp/azurerm"     version = "3.90.0" }`.
        - This states that version `3.90.0` of the `hashicorp/azurerm` provider (plugin) is required.
        - It will be downloaded from the Hashicorp provider registry (https://registry.terraform.io/browse/providers).
    - An argument `required_version = "~> 1.7.0"` which specifies the required version (`1.7.x`) of the `terraform` CLI tool.
  - The `provider` block is used to configure the `azurerm` provider, and contains one nested block `features`:
    - Since the body is empty for the `features` block, default values will be used for the `azurerm` provider.

In [4]:
# !type providers.tf # use this on Windows
!cat providers.tf

# Initialises Terraform providers and sets their version numbers.
# Note:
# 1) We require the provider "hashicorp/azurerm" version "3.90.0" (a specific Azure provider).
# 2) We require the version of Terraform to be "1.7.x".
# 3) We are using the provider "azurerm" (with default features) that we required in (1).

terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "3.90.0"
    }
  }

  required_version = "~> 1.7.0"
}

provider "azurerm" {
  features {}
}

## Initialize a Terraform Project

- The command `terraform init` initializes a Terraform project (i.e. for the current folder).
  - It will download the configured provider `azurerm` (in `providers.tf`) to the folder `.terraform`.
  - It will Store version information about the provider `azurerm` in the file `.terraform.lock.hcl`.

In [5]:
#!rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## Let's view the contents of the file `variables.tf`

- This file defines Terraform variables, where a variable is defined as a name-value pair.
  - Terraform variables can be referenced in other `.tf` files in the same Terraform project (folder).
- In this case only two variables are defined.
  - The variable with name `app_name` and value `tsfn14g00`.
    - In this exercise, I'm using this variable to provide a name for an entire Azure application.
  - The variable with name `location` and value `westeurope`.
    - In this exercise, I'm using this variable to determine what Azure `zone/region` to use for storing my Azure resources.
      - A `zone/region` relates to one of Microsoft's data centers (in this case one that is located in Western Europe).

**Note**

- Change the value of the `app_name` variable from `tsfn14g00` to something unique.
  - This is required since it is used to define URLs in Azure that have to be globally unique.
  - The pattern I have used is:
    - The course code `tsfn14`.
    - Followed by a `g` for `group`.
    - And finally a `number` representing a `group number` (in my case I am using group `00`).

In [6]:
#!type variables.tf # use this on Windows
!cat variables.tf

# Sets global variables for this Terraform project.
# Note:
# - The block type is "variable" since we are defining a Terraform variable.
# - "app_name" and "location" is the names of the Terraform variables.
# - "default" is used to set the value for a Terraform variable.
# - if "default" is omitted, Terraform will ask the user to input the value for the variable during "terraform apply".

variable "app_name" {
  default = "tsfn14g00"
}

variable "location" {
  default = "westeurope"
}

## Let's view the contents of the file `resource-group.tf`

- This file defines an Azure `Resource Group`.
  - A `Resource Group` is used as a `namespace` in which various resources can be logically organized.
  - All resources contained in a `Resource Group` are destroyed when the `Resource Group` is destroyed.
- A resouce is defined using:
  - The **Block Type** `resource`.
  - Followed by a **Block Label** containing:
    - The name of the provider (in this case `azurerm`).
    - Followed by an underscore (`-`).
    - Then the name of the provider's resource you want (in this case `resource_group` for an Azure Resource Group).
  - The second **Block Label** gives this resource block a unique name (`main`), used to identify it in this and other Terraform files.
  - Everything within brackets (`{ }`) is the **Block Body** which contains **Arguments** for configuring the resource.
    - An Azure Resource Group needs a unqiue `name` and `location`.
    - In this case, the variables defined in the file `variables.tf` are used to set these properties.
      - A variable's value is used by preceeding the variable's name with the word `var`, then a dot `.`, and finally the variable's name.
- This file defines an Azure Resource Group
  - With the `name` set to `tsfn14g00` (according to the current value of the variable `app_name` in the file `variables.tf`).
  - And the `location` set to `westeurope` (according to the current value of the variable `location` in the file `variables.tf`).

In [7]:
#!type resource-group.tf #use this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.
# Note:
# - The block type is "resource" since we are defining a resource.
# - The resource we are defining is "azurerm_resource_group" (an Azure Resource Group).
# - "main" is the name used to identify the resource in Terraform as "azurerm_resource_group.main"
# - "name" is the name of the resource group in Azure.
# - "location" is the region in Azure (where the resource group metadata will be stored).
# - Notice how the Terrform variables (defined in the file "variables.tf") are used:
#   - "var.app_name" refers to the variable "app_name" and returns its associated value.
#   - "var.location" refers to the variable "location" and returns its associated value.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}

## Terraform Fmt

- The command `terraform fmt` formats `.tf` files in the current folder (e.g. correct indentation, etc.).

In [8]:
!terraform fmt

## Terraform Validate

- The command `terraform validate` validates `.tf` files in the current folder for correct syntax.

In [9]:
!terraform validate

Success! The configuration is valid.



## Terraform Plan

- The command `terraform plan` show Terraform's execution plan (without executing it) for all `.tf` files in the current directory.
  - All Terraform files (`.tf`) in the current directory will be merged in memory.
  - Then Terraform calculates what changes need to be made to set up the infrastructure (resources) defined in the files.
  - Finally, Terraform outputs an `executation plan` for setting up the infrastructure (on Azure in this case).
    - Note that the `executation plan` isn't executed at this time.

In [10]:
!terraform plan


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_resource_group.main will be created
  + resource "azurerm_resource_group" "main" {
      + id       = (known after apply)
      + location = "westeurope"
      + name     = "tsfn14g00"
    }

Plan: 1 to add, 0 to change, 0 to destroy.

───────────────────────────────────────────────────────────────────────────────

Note: You didn't use the -out option to save this plan, so Terraform can't
guarantee to take exactly these actions if you run "terraform apply" now.


## Terraform Apply

- The command `terraform apply` will apply Terraform's `execution plan` for all `.tf` files in the current directory.
- The current **Terraform state** will be stored in the files `terraform.tfstate` and `.terraform.lock.hcl`.
  - Terraform uses these files to compare the current state of the actual infrastructure with the state declared in the Terraform files.
    - This is how Terraform knows how to update existing infrastructure.

### Note!

The command `terraform apply` can be used:
- Without the `-auto-approve` option
  -  Terraform will ask for a confirmation (`yes` or `no`) before applying.
- With the `-auto-approve` option
  - Terrafowm will **not** ask for any confirmation before applying.

In [11]:
#!terraform apply -auto-approve
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_resource_group.main will be created
  + resource "azurerm_resource_group" "main" {
      + id       = (known after apply)
      + location = "westeurope"
      + name     = "tsfn14g00"
    }

Plan: 1 to add, 0 to change, 0 to destroy.
azurerm_resource_group.main: Creating...
azurerm_resource_group.main: Creation complete after 3s [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]

Apply complete! Resources: 1 added, 0 changed, 0 destroyed.


## List Azure Resource Groups

- The Azure CLI command `az group list -o table` will list existing resoure groups in Azure.
- We see that Terraform has created the Resource Group `tsfn14g00` on Azure.

In [12]:
!az group list -o table

Name       Location    Status
---------  ----------  ---------
tsfn14g00  westeurope  Succeeded


## List Terraform state

- The Terraform CLI command `!terraform state list` will list all Terraform state (resources) contained in the file `terraform.state`.
- We see that Terraform has an Azure Resource group `azurerm_resource_group` identified as `main` in its file `terraform.state`.
  - The full qualifier is `azurerm_resource_group.main` since it was defined in the file `resource-group.tf` as:

    ```bash
    resource "azurerm_resource_group" "main" {
      name     = var.app_name
      location = var.location
    }
    ```

In [13]:
!terraform state list

azurerm_resource_group.main


## Show Terraform state for `azurerm_resource_group.main`

- The Terraform CLI command `terraform state show azurerm_resource_group.main` shows:
  - The definition for the Terraform state `azurerm_resource_group.main` in the file `terraform.state`.

In [14]:
!terraform state show azurerm_resource_group.main

# azurerm_resource_group.main:
resource "azurerm_resource_group" "main" {
    id       = "/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00"
    location = "westeurope"
    name     = "tsfn14g00"
}


## Show all Terraform state

- The Terraform CLI command `terraorm show` will show the definition for **all Terraform state** in the file `terraform.state`.
  - In this case we have only defined one resource (state) in the current Terraform project (folder).

In [15]:
!terraform show

# azurerm_resource_group.main:
resource "azurerm_resource_group" "main" {
    id       = "/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00"
    location = "westeurope"
    name     = "tsfn14g00"
}


## Terraform Destroy

- The Terraform CLI command `terraform destroy` will:
  - Destroy all resources (infrastructure) defined in the current Terraform project (i.e. `.tf` files in the current folder)-
  - Backup the old Terraform state to the file `terraform.tfstate.backup`.
  - Update the file `terraform.tfstate` with the new (current) Terraform state.

### Note!

- The 'terraform destroy' command can be executed:
  - Without `-auto-approve` option
    -  Terraform will ask for a confirmation (`yes` or `no`) before destroying.
  - With `-auto-approve` option
    - Terrafowm will **not** ask for any confirmation before destroying.

In [16]:
!terraform destroy -auto-approve

azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_resource_group.main will be destroyed
  - resource "azurerm_resource_group" "main" {
      - id       = "/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00" -> null
      - location = "westeurope" -> null
      - name     = "tsfn14g00" -> null
      - tags     = {} -> null
    }

Plan: 0 to add, 0 to change, 1 to destroy.
azurerm_resource_group.main: Destroying... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_resource_group.main: Still destroying... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00, 10s elapsed]
azurerm_resource_group.main: Destructi

## List Azure Resource Groups

- After running `terraform destroy` there shouldn't be any Resource Groups in Azure.
- Therefore, the Azure CLI command `az group list -o table` shouldn't return any Resource Groups.

In [17]:
!az group list -o table

## Show all Terraform state

- After running `terraform destroy` the Terraform state should have been updated in the file `terraform.state`.
- Therefore, the Terraform CLI command `terraform show` should,'t return any resources from the file `terraform.state`.

In [18]:
!terraform show

The state file is empty. No resources are represented.
